In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

def ignore_warn(*args, **kwargs):
    pass

In [3]:
data = pd.read_pickle(r'C:\Self-Learning\Kaggle\CustomerSegmentation\online retail\features.pkl')
feature_clusters = pd.read_pickle(r'C:\Self-Learning\Kaggle\CustomerSegmentation\online retail\cluster.pkl')

In [5]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount,Internal
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Yes
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Yes
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Yes
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Yes
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Yes


In [6]:
feature_clusters.head()

,CustomerID,recency,frequency,amount,recency_log,frequency_log,amount_log,clusters_3,clusters_5,clusters_7
0,12346,326.0,1,77183.60,5.786897,0.000000,11.253942,2,3,2
1,12347,2.0,7,4310.00,0.693147,1.945910,8.368693,0,1,5
2,12348,75.0,4,1797.24,4.317488,1.386294,7.494007,2,3,2
3,12349,19.0,1,1757.55,2.944439,0.000000,7.471676,2,0,0
4,12350,310.0,1,334.40,5.736572,0.000000,5.812338,1,2,4


<b>Build Transaction Dataset</b><br>
In order to perform our data in these algorithms, we must first turn them into a sales event table where each product sold will be represented by a column, having its value 1 for when it was sold in that event or zero when not. This will generate a sparse table

In [7]:
items = list(data.Description.unique())

In [8]:
grouped = data.groupby(['InvoiceNo'])

In [11]:
transaction_level = grouped.aggregate(lambda x: tuple(x)).reset_index()[['InvoiceNo', 'Description']]

In [18]:
temp = dict()
for rec in transaction_level.to_dict('records'):
    invoice_num = rec['InvoiceNo']
    items_list = rec['Description']
    transaction_dict = {item:0 for item in items}
    transaction_dict.update({item:1 for item in items if item in items_list})
    temp.update({invoice_num:transaction_dict})
    
new = [v for k,v in temp.items()]
transaction_df = pd.DataFrame(new)

In [19]:
transaction_df

,WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL LANTERN,CREAM CUPID HEARTS COAT HANGER,KNITTED UNION FLAG HOT WATER BOTTLE,RED WOOLLY HOTTIE WHITE HEART.,SET 7 BABUSHKA NESTING BOXES,GLASS STAR FROSTED T-LIGHT HOLDER,HAND WARMER UNION JACK,HAND WARMER RED RETROSPOT,ASSORTED COLOUR BIRD ORNAMENT,...,SET OF 6 RIBBONS PERFECTLY PRETTY,SET OF 6 RIBBONS PARTY,"LETTER ""F"" BLING KEY RING","LETTER ""O"" BLING KEY RING","LETTER ""T"" BLING KEY RING","LETTER ""U"" BLING KEY RING","LETTER ""W"" BLING KEY RING","LETTER ""Z"" BLING KEY RING",PINK CRYSTAL SKULL PHONE CHARM,"PAPER CRAFT , LITTLE BIRDIE"
0,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18529,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
18530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<b>Prune Dataset for frequently purchased items</b><br>
We saw in the earlier on EDA how only a handful of items are responsible for bulk of our sales so we want to prune our dataset to reflect this information. For this we have created a function prune_dataset below, which will help us reduce the size of our dataset based on our requirements. The function can be used for performing three types of pruning:

* Pruning based on <b>percentage of total sales</b>: The parameter total_sales_perc will help us select the number of items that will explain the required percentage of sales. The default value is 50% or 0.5.
*  Pruning based on <b>ranks of items</b>: Another way to perform the pruning is to specify the starting and the ending rank of the items for which we want to prune our dataset.
*  Pruning based on list of features passed to the parameter <b>TopCols</b>.

NOTE:<br> By default, we will only look for transactions which have at least two items, as transactions with only one item are counter to the whole concept of association rule-mining.

In [26]:
transaction_df_copy = transaction_df.copy()
item_counts = transaction_df_copy.sum().sort_values(ascending=False).reset_index()
total_count = sum(transaction_df_copy.sum())
item_counts

,index,0
0,WHITE HANGING HEART T-LIGHT HOLDER,1978
1,REGENCY CAKESTAND 3 TIER,1703
2,JUMBO BAG RED RETROSPOT,1600
3,PARTY BUNTING,1379
4,ASSORTED COLOUR BIRD ORNAMENT,1375
...,...,...
3642,PURPLE CHUNKY GLASS+BEAD NECKLACE,1
3643,PACK 4 FLOWER/BUTTERFLY PATCHES,1
3644,BLOSSOM IMAGES SCRAP BOOK SET,1
3645,PINK FLUFFY CHRISTMAS DECORATION,1


In [30]:
item_counts = item_counts.rename(columns = {item_counts.columns[0]: 'item_name', 
                             item_counts.columns[1]: 'item_count'})
item_counts['item_perc'] = item_counts['item_count']/total_count
item_counts

,item_name,item_count,item_perc
0,WHITE HANGING HEART T-LIGHT HOLDER,1978,0.005101
1,REGENCY CAKESTAND 3 TIER,1703,0.004392
2,JUMBO BAG RED RETROSPOT,1600,0.004126
3,PARTY BUNTING,1379,0.003556
4,ASSORTED COLOUR BIRD ORNAMENT,1375,0.003546
...,...,...,...
3642,PURPLE CHUNKY GLASS+BEAD NECKLACE,1,0.000003
3643,PACK 4 FLOWER/BUTTERFLY PATCHES,1,0.000003
3644,BLOSSOM IMAGES SCRAP BOOK SET,1,0.000003
3645,PINK FLUFFY CHRISTMAS DECORATION,1,0.000003


In [83]:
def prune_dataset(input_df, length_trans=2, total_sales_perc=0.5, start_item=None, end_item=None, TopCols=None):
    
    if 'total_items' in input_df.columns:
        del(input_df['total_items'])
        
    item_counts = input_df.sum().sort_values(ascending=False).reset_index()
    total_count = sum(input_df.sum())
    item_counts.rename(columns={item_counts.columns[0]:'item_name',
                               item_counts.columns[1]:'item_count'}, inplace=True)
    
    if TopCols:
        input_df['total_items'] = input_df[TopCols].sum(axis=1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[TopCols], item_counts[item_counts.item_name.isin(TopCols)]
    elif end_item > start_item:
        selected_items = list(item_counts[start_item:end_item].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis=1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items], item_counts[start_item:end_item]
    else:
        item_counts['item_perc'] = item_counts['item_count']/total_count
        item_counts['total_perc'] = item_counts.item_count.cumsum()
        selected_items = list(item_counts[item_counts['total_perc'] < total_sales_perc].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis=1)
        input_df = input_df[input_df['total_items'] > length_trans]
        del(input_df['total_items'])
        return input_df[selected_items], item_counts[item_counts.total_perc < total_sales_perc]

        
    
    

In [84]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount,Internal
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Yes
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Yes
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Yes
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Yes
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Yes


In [85]:
AmoutSum = data[['Description', 'amount']].groupby(['Description']).amount.sum().sort_values(ascending=False)
inv = data[['Description', 'InvoiceNo']].groupby(['Description']).InvoiceNo.unique().agg(np.size).sort_values(ascending=False)

Top15ev = list(inv[:15].index)

In [86]:
# We use the second option of pruning, by the Top 15th products in sales events.
output_df, item_counts = prune_dataset(input_df=transaction_df, length_trans=2, start_item=0, end_item=15)
print('Total of Sales Amount by the Top 15 Products in Sales Events (Invoice): {:.2f}'.format(AmoutSum[Top15ev].sum()))
print('Number of Sales Events:', output_df.shape[0])
print('Number of Products:', output_df.shape[1])

item_counts

Total of Sales Amount by the Top 15 Products in Sales Events (Invoice): 778377.21
Number of Sales Events: 4664
Number of Products: 15


,item_name,item_count
0,WHITE HANGING HEART T-LIGHT HOLDER,1978
1,REGENCY CAKESTAND 3 TIER,1703
2,JUMBO BAG RED RETROSPOT,1600
3,PARTY BUNTING,1379
4,ASSORTED COLOUR BIRD ORNAMENT,1375
5,LUNCH BAG RED RETROSPOT,1289
6,SET OF 3 CAKE TINS PANTRY DESIGN,1146
7,POSTAGE,1099
8,JUMBO BAG VINTAGE DOILY,1080
9,LUNCH BAG BLACK SKULL.,1052


### Import Orange Module

In [90]:
import Orange
from Orange.data import Domain, DiscreteVariable, ContinuousVariable
from orangecontrib.associate.fpgrowth import *

### Association Rule Mining with FP Growth

In [91]:
input_assoc_rules = output_df

In [94]:
# Defined the data domain by specifying each variable as a DiscreteVariable having values as (0, 1)
domain_transac = Domain([DiscreteVariable.make(name=item, values=['0','1']) for item in input_assoc_rules.columns])

In [97]:
# Then using this domain, we created our Table structure for our data
data_trans = Orange.data.Table.from_numpy(domain=domain_transac, X=input_assoc_rules.values, Y=None)

In [99]:
# Coding our input so that the entire domain is represented as binary variables
data_trans_en, mapping = OneHot.encode(data_trans, include_class=True)

array([[ True, False,  True, ..., False,  True, False],
       [False,  True,  True, ..., False,  True, False],
       [ True, False,  True, ..., False,  True, False],
       ...,
       [ True, False,  True, ..., False,  True, False],
       [False,  True,  True, ..., False,  True, False],
       [ True, False,  True, ...,  True,  True, False]])

In [106]:
# Creating our rules
support = 0.01
print("num of required transactions = ", int(input_assoc_rules.shape[0]*support))
num_trans = input_assoc_rules.shape[0]*support
itemsets = dict(frequent_itemsets(data_trans_en, support))
print("Item set size: ", len(itemsets))

num of required transactions =  46
Item set size:  663273


In [107]:
confidence = 0.6
rules_df = pd.DataFrame()


{frozenset({14}): 4165,
 frozenset({22}): 3967,
 frozenset({14, 22}): 3503,
 frozenset({24}): 3896,
 frozenset({14, 24}): 3491,
 frozenset({22, 24}): 3349,
 frozenset({14, 22, 24}): 2974,
 frozenset({12}): 3872,
 frozenset({12, 14}): 3435,
 frozenset({12, 22}): 3284,
 frozenset({12, 14, 22}): 2881,
 frozenset({12, 24}): 3252,
 frozenset({12, 14, 24}): 2904,
 frozenset({12, 22, 24}): 2783,
 frozenset({12, 14, 22, 24}): 2464,
 frozenset({26}): 3842,
 frozenset({12, 26}): 3174,
 frozenset({14, 26}): 3393,
 frozenset({12, 14, 26}): 2782,
 frozenset({22, 26}): 3224,
 frozenset({12, 22, 26}): 2652,
 frozenset({14, 22, 26}): 2806,
 frozenset({12, 14, 22, 26}): 2290,
 frozenset({24, 26}): 3178,
 frozenset({12, 24, 26}): 2637,
 frozenset({14, 24, 26}): 2820,
 frozenset({12, 14, 24, 26}): 2331,
 frozenset({22, 24, 26}): 2690,
 frozenset({12, 22, 24, 26}): 2218,
 frozenset({14, 22, 24, 26}): 2358,
 frozenset({12, 14, 22, 24, 26}): 1937,
 frozenset({8}): 3769,
 frozenset({8, 12}): 3106,
 frozenset

In [113]:
rules = [(P, Q, supp, conf) for P, Q, supp, conf in association_rules(itemsets, confidence) if len(Q) == 1]


In [116]:
names = {item: '{}={}'.format(var.name, val) for item, var, val in OneHot.decode(mapping, data_trans, mapping)}

In [118]:
eligible_ante = [v for k,v in names.items() if v.endswith("1")]

In [123]:
N = input_assoc_rules.shape[0]
rule_stats = list(rules_stats(rules, itemsets, N))

In [143]:
# rule_stats[0]
names[next(iter(rule_stats[0][1]))][:-2]

'WHITE HANGING HEART T-LIGHT HOLDER'

In [145]:
rule_list_df = []

for ex_rule_frm_rule_stat in rule_stats:
    ante = ex_rule_frm_rule_stat[0]
    cons = ex_rule_frm_rule_stat[1]
    named_cons = names[next(iter(cons))]
    if named_cons in eligible_ante:
        rule_lhs = [names[i][:-2] for i in iter(ante) if names[i] in eligible_ante]
        ante_rule = ', '.join(rule_lhs)
        if ante_rule and len(rule_lhs)>1:
            rule_dict = {
                'support': ex_rule_frm_rule_stat[2],
                'confidence': ex_rule_frm_rule_stat[3],
                'coverage': ex_rule_frm_rule_stat[4],
                'strength': ex_rule_frm_rule_stat[5],
                'lift': ex_rule_frm_rule_stat[6],
                'leverage': ex_rule_frm_rule_stat[7],
                'antecedent': ante_rule,
                'consequent': named_cons[:-2] }
            rule_list_df.append(rule_dict)

rules_df = pd.DataFrame(rule_list_df)

In [148]:
rules_df.shape

(25247, 8)

In [147]:
print("Raw rules data frame of {} rules generated".format(rules_df.shape[0]))
if not rules_df.empty:
    pruned_rules_df = rules_df.groupby(['antecedent','consequent']).max().reset_index()
else:
    print("Unable to generate any rule")

pruned_rules_df

Raw rules data frame of 25247 rules generated


,antecedent,consequent,support,confidence,coverage,strength,list,leverage
0,"ASSORTED COLOUR BIRD ORNAMENT, LUNCH BAG SUKI ...",LUNCH BAG RED RETROSPOT,48,0.600000,0.017153,14.575000,2.400000,0.006003
1,"JUMBO BAG RED RETROSPOT, JUMBO BAG VINTAGE DOI...",LUNCH BAG RED RETROSPOT,75,0.761905,0.023156,18.507937,3.047619,0.010292
2,"JUMBO BAG RED RETROSPOT, JUMBO BAG VINTAGE DOI...",LUNCH BAG VINTAGE DOILY,78,0.774194,0.023156,14.819672,3.994291,0.012360
3,"JUMBO BAG RED RETROSPOT, JUMBO BAG VINTAGE DOI...",LUNCH BAG RED RETROSPOT,63,0.847458,0.017796,20.821429,3.389831,0.009059
4,"JUMBO BAG RED RETROSPOT, JUMBO BAG VINTAGE DOI...",LUNCH BAG BLACK SKULL.,51,0.602410,0.018225,11.192771,3.024369,0.007305
...,...,...,...,...,...,...,...,...
134,"PARTY BUNTING, LUNCH BAG VINTAGE DOILY",SPOTTY BUNTING,64,0.680556,0.021655,11.742857,3.861449,0.009906
135,"WHITE HANGING HEART T-LIGHT HOLDER, LUNCH BAG ...",LUNCH BAG SUKI DESIGN,54,0.627907,0.018439,12.473684,3.089196,0.007830
136,"WHITE HANGING HEART T-LIGHT HOLDER, LUNCH BAG ...",LUNCH BAG RED RETROSPOT,62,0.626263,0.021226,14.575000,2.505051,0.007987
137,"WHITE HANGING HEART T-LIGHT HOLDER, LUNCH BAG ...",LUNCH BAG SUKI DESIGN,54,0.662162,0.018225,12.810811,3.257726,0.007874


In [149]:
dw = pd.options.display.max_colwidth
pd.options.display.max_colwidth = 100

In [164]:
(rules_df[['consequent', 'antecedent', 'support','confidence','list']].\
 groupby(['consequent', 'antecedent']).first()
                                      .reset_index()
                                      .sort_values(['confidence', 'support', 'list'],
                                                   ascending=False))

,consequent,antecedent,support,confidence,list
20,JUMBO BAG VINTAGE DOILY,"JUMBO BAG RED RETROSPOT, LUNCH BAG RED RETROSP...",60,0.909091,4.520256
133,LUNCH BAG VINTAGE DOILY,"LUNCH BAG RED RETROSPOT, JUMBO BAG VINTAGE DOI...",49,0.890909,4.596460
134,LUNCH BAG VINTAGE DOILY,"LUNCH BAG RED RETROSPOT, JUMBO BAG VINTAGE DOI...",48,0.888889,4.586037
131,LUNCH BAG VINTAGE DOILY,"LUNCH BAG RED RETROSPOT, JUMBO BAG VINTAGE DOI...",48,0.872727,4.502655
21,JUMBO BAG VINTAGE DOILY,"JUMBO BAG RED RETROSPOT, LUNCH BAG SUKI DESIGN...",48,0.872727,4.339446
...,...,...,...,...,...
49,LUNCH BAG BLACK SKULL.,"WHITE HANGING HEART T-LIGHT HOLDER, LUNCH BAG ...",54,0.600000,3.012271
46,LUNCH BAG BLACK SKULL.,"LUNCH BAG SUKI DESIGN , LUNCH BAG VINTAGE DOIL...",51,0.600000,3.012271
35,LUNCH BAG BLACK SKULL.,"JUMBO BAG RED RETROSPOT, LUNCH BAG SUKI DESIGN...",48,0.600000,3.012271
100,LUNCH BAG SUKI DESIGN,"JUMBO BAG RED RETROSPOT, LUNCH BAG BLACK SKUL...",48,0.600000,2.951899
